In [2]:
from paddlenlp import Taskflow
import paddle
import pandas as pd
import ast
from datetime import datetime
import re

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


/root/miniconda3/envs/paddle_env/lib/python3.8/site-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
/root/miniconda3/envs/paddle_env/lib/python3.8/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
text = ''
# files_path = r'../../temp/甘肃省_学习考察.xlsx'
files_path = r'../../temp/全对测试集_学习考察.xlsx'
# paddle.device.set_device("gpu")
original_data = pd.read_excel(files_path)
data = original_data.copy()

In [4]:
def check_text(news_text):
    
    # 检查全文是否包含 "市长" 或 "省长"
    if "市长" in news_text:
        return True
    if "省长" in news_text:
        return True
    
    # 将新闻文本按照句子进行分割
    sentences = re.split('[。！？；\n]', news_text)
    
    # 遍历每个句子
    for sentence in sentences:
        sentence = sentence.strip()
        
        # 检查 "市委" 和 "书记"
        if "市委" in sentence and "书记" in sentence:
            shiwei_positions = [m.start() for m in re.finditer("市委", sentence)]
            shuji_positions = [m.start() for m in re.finditer("书记", sentence)]
            for pos_shiwei in shiwei_positions:
                for pos_shuji in shuji_positions:
                    end_pos_shiwei = pos_shiwei + len("市委")
                    end_pos_shuji = pos_shuji + len("书记")
                    if pos_shiwei <= pos_shuji:
                        distance = pos_shuji - end_pos_shiwei
                    else:
                        distance = pos_shiwei - end_pos_shuji
                    if distance <= 5:
                        return True
        
        # 检查 "省委" 和 "书记"
        if "省委" in sentence and "书记" in sentence:
            shengwei_positions = [m.start() for m in re.finditer("省委", sentence)]
            shuji_positions = [m.start() for m in re.finditer("书记", sentence)]
            for pos_shengwei in shengwei_positions:
                for pos_shuji in shuji_positions:
                    end_pos_shengwei = pos_shengwei + len("省委")
                    end_pos_shuji = pos_shuji + len("书记")
                    if pos_shengwei <= pos_shuji:
                        distance = pos_shuji - end_pos_shengwei
                    else:
                        distance = pos_shengwei - end_pos_shuji
                    if distance <= 5:
                        return True
    return False


In [5]:
def process_content(x):
    # 防止长度过长
    if len(x) > 1500:
        return x
    return ','.join(x)

data['content'] = data['content'].apply(ast.literal_eval)
data['content'] = data['content'].apply(process_content)
data['is_match'] = data['content'].apply(check_text)
filtered_data = data[data['is_match'] == True]

In [8]:
data

,content,is_match
0,3月29日，蚌埠市党政代表团一行在合肥晶合集成电路有限公司考察，合肥市委副书记、市长凌云陪同...,True
1,6月22日至23日，市委书记黄晓武率市党政代表团赴广东省潮州、梅州两市学习考察。黄晓武在考察...,True
2,​ 3月26日至30日，市委副书记李明在市委副秘书长张信鹏，市委党校、市委办公室、市农委等单...,True
3,8月8日，市委常委、常务副市长杨东坡率部分市直单位及县区负责同志等前往阜阳市，考察学习大气污...,True
4,4月23日至24日，市人民政府副市长孙军率领市发改、住建、城管、规划等部门负责人一行赴黄山市...,True
...,...,...
147,8月4日至6日，市委副书记、市长毛宏芳率领嘉兴市代表团赴西藏自治区那曲市色尼区等地考察，看望...,True
148,昨天，泰安市党政考察团一行参观了南湖革命纪念馆，瞻仰了南湖红船。今天，考察团一行将赴桐乡考察...,True
149,张兵简要介绍了嘉兴的特点和发展情况，他说，嘉兴是中国共产党的梦想起航地，位于长三角中心腹地，...,True
150,陈奕君对金华市党政代表团的到来表示欢迎。她说，台州金华两地长期以来交流合作十分密切。近年来，...,True


In [6]:
len(filtered_data)

152

In [9]:
# 关系抽取
schema = {'考察人姓名': ['考察人职位', '考察人所属城市', '所考察的城市']}
ie = Taskflow('information_extraction', schema=schema, batch_size=8)

[2024-09-20 16:40:33,759] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load '/root/.paddlenlp/taskflow/information_extraction/uie-base'.


In [10]:
# schema = {'考察触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '访问触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人'],
#           '学习触发词': ['来访城市', '受访城市', '来访人', '职位', '接访人']
#           }

# 批量处理数据
results = [ie(content) for content in filtered_data['content']]

/root/miniconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/tokenizer_utils_base.py:3023: UserWarning: Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  warnings.warn(


In [11]:
results

[[{'考察人姓名': [{'text': '郑东涛',
     'start': 110,
     'end': 113,
     'probability': 0.26653646470312964},
    {'text': '凌云', 'start': 42, 'end': 44, 'probability': 0.3250633844374988},
    {'text': '汪莹纯',
     'start': 57,
     'end': 60,
     'probability': 0.43704927382457726},
    {'text': '胡启望',
     'start': 114,
     'end': 117,
     'probability': 0.2585018952554563},
    {'text': '汪莹纯',
     'start': 988,
     'end': 991,
     'probability': 0.8126062425579335},
    {'text': '郑东涛',
     'start': 1151,
     'end': 1154,
     'probability': 0.4762047488722345},
    {'text': '胡启望',
     'start': 1155,
     'end': 1158,
     'probability': 0.4625451876112763},
    {'text': '杨森',
     'start': 1148,
     'end': 1150,
     'probability': 0.3998061462325069},
    {'text': '高萍',
     'start': 1159,
     'end': 1161,
     'probability': 0.3169452880621293},
    {'text': '汪莹纯',
     'start': 1098,
     'end': 1101,
     'probability': 0.3949390312573442},
    {'text': '汪莹纯',
     'start

In [12]:
# 将结果展开并格式化为 DataFrame 的行
formatted_results = []
for i, content in enumerate(data['content']):
    for item in results[i]:
        for person in item.get('考察人姓名', []):
            result_row = {
                'content': content,
                '人物': person.get('text', ''),
                '职位': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('考察人职位', [])]),
                '所属城市': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('考察人所属城市', [])]),
                '考察城市': ','.join([relation.get('text', '') for relation in person.get('relations', {}).get('所考察的城市', [])])
            }
            formatted_results.append(result_row)
            
# 用于存储去重后的结果
unique_results = {}

# 遍历每一条记录，按人物去重，并保留信息最多的那一项
for result in formatted_results:
    person = result['人物']
    
    # 如果该人物尚未在字典中，或当前记录信息更多，则更新字典
    if person not in unique_results or (
        len(result['职位']) + len(result['所属城市']) + len(result['考察城市']) >
        len(unique_results[person]['职位']) + len(unique_results[person]['所属城市']) + len(unique_results[person]['考察城市'])
    ):
        unique_results[person] = result

# 创建 DataFrame
results_df = pd.DataFrame(list(unique_results.values()))

current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# 保存为 CSV 文件
output_path = f'{current_time}_extracted_relations.csv'
results_df.to_csv(output_path, index=False, encoding='utf-8-sig')